In [14]:
from rank_bm25 import BM25Okapi
import numpy as np
import csv
from tqdm import tqdm
from konlpy.tag import Mecab

In [15]:
mecab = Mecab("C:\mecab\mecab-ko-dic")
def mecab_tokenizer(sent):
    return mecab.morphs(sent)

document

In [17]:
doc_id = []
corpus = []

with open('./aihub/collection.tsv', 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.split('||', 1)
        docid = parts[0].strip()
        corpus_ = parts[1].strip()
        doc_id.append(docid)
        corpus.append(corpus_)

In [18]:
len(doc_id), len(corpus)

(124535, 124535)

tokenizer

In [19]:
tokenized_corpus = [mecab_tokenizer(doc) for doc in tqdm(corpus)]

  0%|          | 0/124535 [00:00<?, ?it/s]

100%|██████████| 124535/124535 [02:39<00:00, 782.87it/s] 


In [20]:
k1 = 1.2
b = 0.75
bm25 = BM25Okapi(tokenized_corpus, k1=k1, b = b)

쿼리

In [21]:
questions = {}

with open('./aihub/questions.tsv', 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.split('\t', 1)
        qid = parts[0].strip()
        question = parts[1].strip()
        if qid in questions.keys():
            print('1')
        else:
            questions[qid] = question


In [22]:
len(questions)

252689

정답

In [23]:
test_qids = []
test_docid = []

with open('./aihub/test/qrels_test.tsv', 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.split('\t', 1)
        qid = parts[0].strip()
        docid = parts[1].strip()
        test_qids.append(qid)
        test_docid.append(docid)

In [24]:
len(test_qids), len(test_docid)

(3000, 3000)

test

In [25]:
recall1 = 0
recall5 = 0
recall10 = 0
recall20 = 0


for idx, test_qid in tqdm(enumerate(test_qids), total = len(test_qids)):
    query = questions[test_qid]
    answer = test_docid[idx]

    tokenized_query = mecab_tokenizer(query)
    doc_scores = bm25.get_scores(tokenized_query)
    
    top_indices = np.argsort(doc_scores)[-100:][::-1]

    for i, idx in enumerate(top_indices):
        if i < 1 and answer == doc_id[idx]:
            recall1 += 1
            recall5 += 1
            recall10 += 1
            recall20 += 1
            break
        elif i < 5 and answer == doc_id[idx]:
            recall5 += 1
            recall10 += 1
            recall20 += 1
            break
        elif i < 10 and answer == doc_id[idx]:
            recall10 += 1
            recall20 += 1
            break
        elif i < 20 and answer == doc_id[idx]:
            recall20 += 1
            break

100%|██████████| 3000/3000 [1:27:53<00:00,  1.76s/it]


In [26]:
print(recall1 / len(test_qids))
print(recall5 / len(test_qids))
print(recall10 / len(test_qids))
print(recall20 / len(test_qids))

0.6346666666666667
0.866
0.9033333333333333
0.9396666666666667
